<a href="https://colab.research.google.com/github/EvagAIML/014-NLP-Model-v1/blob/main/Copy_of_Medical_Diagnostic_Tool_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Medical Diagnostic Assistant: AI-Powered Question Answering

## Problem Statement

### Business Context
The healthcare industry is rapidly evolving, with professionals facing increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. The need for quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.

Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care.

### Objective
As an AI specialist, the task is to develop a **Retrieval-Augmented Generation (RAG)** solution using renowned medical manuals to address healthcare challenges. The objective is to **understand** issues like information overload, **apply** AI techniques to streamline decision-making, **analyze** its impact on diagnostics and patient outcomes, **evaluate** its potential to standardize care practices, and **create** a functional prototype demonstrating its feasibility and effectiveness.

**Common Questions to Answer**

1. **Diagnostic Assistance**: "What are the common symptoms and treatments for pulmonary embolism?"
2. **Surgical Inquiry**: "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
3. **Drug Information**: "Can you provide the trade names of medications used for treating hypertension?"
4. **Treatment Plans**: "What are the first-line options and alternatives for managing rheumatoid arthritis?"
5. **Dermatology**: "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
6. **Neurology**: "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
7. **Emergency Care**: "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
8. **Specialty Knowledge**: "What are the diagnostic steps for suspected endocrine disorders?"
9. **Critical Care Protocols**: "What is the protocol for managing sepsis in a critical care unit?"

### Data Description
The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs.

The manual is provided as a PDF: `014-NLP-PROJ-medical_diagnosis_manual_19.pdf`

## Part 1: Setup and Initialization
Installing necessary libraries and setting up the environment.

In [ ]:
# Installation for GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.28 --force-reinstall --no-cache-dir -q 2>/dev/null

# Installing other dependencies
!pip install huggingface_hub==0.35.3 pandas==2.2.2 tiktoken==0.12.0 pymupdf==1.26.5 langchain==0.3.27 langchain-community==0.3.31 chromadb==1.1.1 sentence-transformers==5.1.1 numpy==1.26.4 -q 2>/dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 31.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 244.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 302.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 244.9 MB/s eta 0:00:00


**Note:** Please restart the runtime after installation.

In [ ]:
import os
import pandas as pd
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

# Define the questions globally
questions = [
    "What are the common symptoms and treatments for pulmonary embolism?",
    "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?",
    "Can you provide the trade names of medications used for treating hypertension?",
    "What are the first-line options and alternatives for managing rheumatoid arthritis?",
    "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?",
    "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?",
    "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?",
    "What are the diagnostic steps for suspected endocrine disorders?",
    "What is the protocol for managing sepsis in a critical care unit?"
]

## Part 2: Question Answering using LLM (Vanilla)
Here we load the LLM and ask the 5 questions without any external context to establish a baseline.

In [ ]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q4_K_M.gguf"

judge_model_name = "TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF"
judge_basename = "tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf"

model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

judge_path = hf_hub_download(
    repo_id=judge_model_name,
    filename=judge_basename
)

# Initialize LLM
llm = Llama(
    model_path=model_path,
    n_ctx=4096,        # Context window
    n_gpu_layers=-1,   # Offload all to GPU suitable for T4
    n_batch=512,
    verbose=True # Changed to True to avoid fileno error
)

# Initialize Judge LLM
judge_llm = Llama(
    model_path=judge_path,
    n_ctx=2048,
    n_gpu_layers=-1,
    verbose=True # Changed to True to avoid fileno error
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 
AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
def generate_response(query, max_tokens=256, temperature=0.1, top_p=0.95, top_k=50, system_prompt=None):
    if system_prompt:
        prompt = f"[INST] {system_prompt}\n{query} [/INST]"
    else:
        prompt = f"[INST] {query} [/INST]"

    model_output = llm(
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k
    )
    return model_output['choices'][0]['text'].strip()

vanilla_results = []
for i, q in enumerate(questions, 1):
    print(f"--- Question {i} ---")
    print(f"Q: {q}")
    ans = generate_response(q)
    print(f"A: {ans}\n")
    vanilla_results.append(ans)

--- Question 1 ---
Q: What are the common symptoms and treatments for pulmonary embolism?
A: A pulmonary embolism (PE) is a serious condition that occurs when a blood clot or other foreign substance travels to the lungs and blocks one or more of the arteries in the lungs. This can lead to decreased blood flow, oxygen levels in the body, and potentially life-threatening complications. Here are some common symptoms of pulmonary embolism:

1. Shortness of breath: This is often the first symptom of a PE. You may feel like you're unable to get enough air or that you're breathing too hard for the level of activity you're doing.
2. Chest pain: This can be described as sharp, stabbing, or crushing pain in the chest, especially when taking deep breaths or coughing.
3. Rapid heart rate: Your heart may beat faster than normal, which can make it harder for your body to get enough oxygen.
4. Coughing up blood: In some cases, you may cough up blood or bloody mucus.
5. Swelling: You may experience sw

Llama.generate: prefix-match hit


A: Appendicitis is a medical condition characterized by inflammation of the appendix, a small tube-shaped organ located in the lower right side of the abdomen. The following are common symptoms of appendicitis:

1. Abdominal pain: The pain typically starts as a mild discomfort around the navel area and then shifts to the lower right side of the abdomen. The pain may be constant or intermittent, and it may worsen with movement, coughing, or sneezing.
2. Loss of appetite: People with appendicitis often lose their appetite due to abdominal discomfort.
3. Nausea and vomiting: Vomiting is a common symptom of appendicitis, especially in advanced cases.
4. Fever: A fever above 100.4°F (38°C) may be present.
5. Diarrhea or constipation: Some people with appendicitis experience diarrhea, while others have constipation.
6. Abdominal swelling: The abdomen may become swollen and tender to the touch.
7. Inability to pass gas: Passing gas can provide

--- Question 3 ---
Q: Can you provide the trade 

Llama.generate: prefix-match hit


A: Yes, I can provide a list of some trade names of medications commonly used to treat hypertension (high blood pressure). Please note that this is not an exhaustive list and there are many other brand names for different classes and combinations of antihypertensive drugs. Here are some examples:

1. Thiazide diuretics: Diuril (chlorothiazide), Maxzide (triamterene/hydrochlorothiazide), and Diovan HCT (valsartan/hydrochlorothiazide)
2. Beta-blockers: Tenormin (atenolol), Lopressor (metoprolol), and Toprol XL (metoprolol succinate)
3. ACE inhibitors: Zestril (lisinopril), Monopress (fosinopril), and Lotensin (benazepril)
4. ARBs (Angiotensin II Receptor Blockers): Cozaar (losartan), Avapro (irbesartan), and Valsartan
5. Calcium channel blockers: Procardia XL (nifedipine

--- Question 4 ---
Q: What are the first-line options and alternatives for managing rheumatoid arthritis?


...

A: If a person has fractured their leg during a hiking trip, it is important that they receive proper medical attention as soon as possible. Here are some necessary precautions and treatment steps:

1. Assess the situation: Check the severity of the injury and assess the overall condition of the person. If the fracture is open or the person is in severe pain, distress, or unable to move, do not attempt to move them without professional help. Call for emergency medical assistance immediately.

2. Provide first aid: If the person's condition allows, provide basic first aid such as cleaning and dressing any wounds, applying a sterile bandage or splint, and administering pain relief medication if available. Do not try to realign the bone or apply excessive pressure to the fracture.

3. Immobilize the leg: Use a makeshift splint or sling to immobilize the leg as best as possible to prevent further injury or movement of the fractured bone. This will help reduce pain and swelling, and prevent

Llama.generate: prefix-match hit


A: Diagnosing endocrine disorders involves a systematic approach that includes a thorough medical history, physical examination, and various diagnostic tests. Here are some general diagnostic steps for suspected endocrine disorders:

1. Medical History: Obtaining a detailed medical history is essential to understand the symptoms, duration, and potential causes of the disorder. This may include questions about family history, lifestyle factors, medications, and past medical conditions.
2. Physical Examination: A physical examination can help identify any signs or symptoms that may be indicative of an endocrine disorder. This may include checking for signs of weight gain or loss, skin changes, growth abnormalities, or other specific findings related to the suspected disorder.
3. Laboratory Tests: Various laboratory tests can help diagnose endocrine disorders. These may include:
   - Blood tests: To measure hormone levels and evaluate organ function. For example, a fasting blood glucose l

Llama.generate: prefix-match hit


A: Sepsis is a life-threatening condition caused by the body's response to an infection, and managing it effectively in a critical care unit requires a multidisciplinary approach. Here are some key steps in the protocol for managing sepsis in a critical care unit:

1. Early recognition and diagnosis: Identify sepsis early by recognizing the signs and symptoms, such as fever, chills, rapid heart rate, rapid breathing, confusion, and low blood pressure. Use the Sequential Organ Failure Assessment (SOFA) score or Quick Sequential Organ Failure Assessment (qSOFA) to help identify patients at risk of developing sepsis.
2. Resuscitation: Provide early and adequate fluid resuscitation to maintain adequate tissue perfusion and organ function. Use intravenous fluids, such as normal saline or lactated Ringer's solution, to replace lost fluids and maintain adequate blood pressure.
3. Antibiotics: Administer broad-spectrum antibiotics as soon as possible to cover potential pathogens. The choice of

In [ ]:
def generate_response(query, max_tokens=256, temperature=0.1, top_p=0.95, top_k=50, system_prompt=None):
    if system_prompt:
        prompt = f"[INST] {system_prompt}\n{query} [/INST]"
    else:
        prompt = f"[INST] {query} [/INST]"

    model_output = llm(
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k
    )
    return model_output['choices'][0]['text'].strip()

### Baseline Answers (Vanilla)

In [ ]:
vanilla_results = []
for i, q in enumerate(questions, 1):
    print(f"--- Question {i} ---")
    print(f"Q: {q}")
    ans = generate_response(q)
    print(f"A: {ans}\n")
    vanilla_results.append(ans)

--- Question 1 ---
Q: What are the common symptoms and treatments for pulmonary embolism?


Llama.generate: prefix-match hit


A: A pulmonary embolism (PE) is a serious condition that occurs when a blood clot or other foreign substance travels to the lungs and blocks one or more of the arteries supplying the lungs with blood. This can lead to decreased oxygen supply to the body, which can be life-threatening if not treated promptly. Here are some common symptoms of pulmonary embolism:

1. Shortness of breath: This is often the first and most prominent symptom of PE. The person may feel like they can't get enough air or that they are breathing too hard for the level of activity they are engaged in.
2. Chest pain: PE can cause sharp, stabbing chest pain that worsens with deep breaths or physical activity. The pain may be described as feeling like a heavy weight on the chest.
3. Rapid heart rate: PE can cause an increase in heart rate, which can lead to palpitations or a racing heart.
4. Coughing up blood: In some cases, PE can cause coughing up blood or bloody sputum.
5. Swelling: PE can cause swelling in the le

Llama.generate: prefix-match hit


A: Appendicitis is a medical condition characterized by inflammation of the appendix, a small tube-shaped organ located in the lower right abdomen. The following are common symptoms of appendicitis:

1. Abdominal pain: The pain usually starts around the navel area and then shifts to the lower right side of the abdomen. It may be constant or come and go, and it can range from mild to severe.
2. Loss of appetite: People with appendicitis often lose their appetite due to abdominal pain and discomfort.
3. Nausea and vomiting: Vomiting is a common symptom of appendicitis, especially if the inflammation is severe.
4. Fever: A fever above 100.4°F (38°C) may be present in some cases.
5. Diarrhea or constipation: Some people with appendicitis experience diarrhea, while others have constipation.
6. Abdominal swelling: The abdomen may become swollen and tender to the touch.
7. Inability to pass gas or stool: A blockage in the intestine can cause

--- Question 3 ---
Q: Can you provide the trade na

...

A: If a person has fractured their leg during a hiking trip, it is important that they receive proper medical attention as soon as possible. Here are some necessary precautions and treatment steps:

1. Assess the situation: Check the severity of the injury and assess the overall condition of the person. If the fracture is open or the person is in severe pain, distress, or unable to move, do not attempt to move them without professional help. Call for emergency medical assistance.

2. Provide first aid: If the person's condition is stable, provide first aid by immobilizing the leg using a splint or a makeshift sling. Use materials available on hand such as sticks, clothing, or backpack straps to create a splint. Be sure not to move the leg excessively and avoid putting weight on it.

3. Keep the person warm: Hypothermia can be a risk in cold environments, so keep the person warm by covering them with blankets or insulating materials.

4. Provide hydration and nutrition: Ensure that the 

Llama.generate: prefix-match hit


A: 1. Thorough Clinical Evaluation: A comprehensive clinical evaluation is essential to identify any symptoms suggestive of an endocrine disorder. This may include assessing medical history, family history, and performing a physical examination.

2. Laboratory Tests: Various laboratory tests are used to diagnose endocrine disorders. These tests may include:
   - Hormone level measurements: Measuring the levels of specific hormones in the blood or urine can help identify an endocrine disorder. For example, measuring insulin levels for diabetes mellitus or thyroid function tests for thyroid disorders.
   - Imaging studies: Imaging techniques such as X-rays, CT scans, MRI, and ultrasound may be used to visualize glands and organs involved in endocrine functions.
   - Genetic testing: In some cases, genetic testing may be necessary to identify inherited endocrine disorders.

3. Diagnostic Procedures: Certain diagnostic procedures can help confirm a suspected endocrine disorder. These inclu

Llama.generate: prefix-match hit


A: Sepsis is a life-threatening condition caused by the body's response to an infection, and managing it effectively in a critical care unit requires a multidisciplinary approach. Here are some key steps in the protocol for managing sepsis in a critical care unit:

1. Early recognition and diagnosis: Identify sepsis early by recognizing signs and symptoms such as fever, chills, tachycardia, tachypnea, altered mental status, and lactate levels. Use validated scoring systems like the Sequential Organ Failure Assessment (SOFA) score or Quick Sequential [Sepsis-related] Organ Failure Assessment (qSOFA) to help identify patients at risk for sepsis.
2. Immediate resuscitation: Start fluid resuscitation with isotonic crystalloids to maintain adequate tissue perfusion and organ function. Target a mean arterial pressure of 65 mmHg or higher and a central venous oxygen saturation (ScvO2) of 70% or higher.
3. Antibiotics: Administer broad-spectrum antibiotics as soon as possible based on culture 

### Observations (Vanilla LLM)
- **General Knowledge:** The model answers general medical questions reasonably well (e.g., symptoms of PE) based on its training data.
- **Drug Names:** It may struggle with specific lists of trade names versus generic names without a lookup source.
- **Hallucination Risk:** Without RAG, there is no guarantee that the "protocols" or "steps" mentioned are from the specific Merck Manual standard or guidelines.

## Part 3: LLM with Prompt Engineering & Parameter Tuning
We will test 5 different combinations of prompts and hyperparameters to observe how the style and quality of answers change.

In [ ]:
# Define 5 Configurations
configs = [
    {"name": "High Creativity", "temp": 0.9, "sys": "You are a helpful medical assistant.", "top_k": 100},
    {"name": "Strict Professional", "temp": 0.0, "sys": "You are a concise, professional doctor. Answer only what is asked.", "top_k": 10},
    {"name": "Chain of Thought", "temp": 0.3, "sys": "Think step-by-step. Explain the reasoning before giving the final answer.", "top_k": 40},
    {"name": "ELI5", "temp": 0.7, "sys": "Explain like I am 5 years old.", "top_k": 50},
    {"name": "Balanced", "temp": 0.4, "sys": "You are a knowledgeable assistant. Provide detailed clinical information.", "top_k": 50}
]

# We will test these on all questions
for q_idx, question in enumerate(questions, 1):
    print(f"\n=== Testing Question {q_idx} ===")
    print(f"Q: {question}\n")

    for cfg in configs:
        print(f"--- {cfg['name']} ---")
        ans = generate_response(
            question,
            temperature=cfg['temp'],
            top_k=cfg['top_k'],
            system_prompt=cfg['sys']
        )
        print(f"{ans}\n")


=== Testing Question 1 ===
Q: What are the common symptoms and treatments for pulmonary embolism?

--- High Creativity ---


Llama.generate: prefix-match hit


A pulmonary embolism (PE) is a serious condition where a blood clot or other foreign substance travels to the lungs, blocking the flow of blood in the pulmonary arteries. This can lead to decreased oxygen supply to the body and potentially life-threatening complications. Here are some common symptoms and treatments for PE:

Symptoms:
1. Shortness of breath: This is one of the most common symptoms of PE. The person may feel like they cannot get enough air or that they are breathing too hard for the level of activity they are doing.
2. Chest pain: PE can cause sharp, stabbing chest pain that worsens with deep breaths or physical activity.
3. Rapid heart rate: PE can lead to a rapid, pounding heartbeat.
4. Coughing up blood: In some cases, PE can cause coughing up blood or blood-tinged sputum.
5. Swelling: PE can cause swelling in the legs, ankles, and arms due to the backup of blood in the veins.
6. Lightheadedness or fainting: PE can cause feelings of lightheadedness or even fain

--- S

Llama.generate: prefix-match hit


A pulmonary embolism (PE) is a blockage in the arteries of the lungs, usually caused by a blood clot that travels from elsewhere in the body. Common symptoms include:

1. Shortness of breath
2. Chest pain or discomfort, often worsening with deep breathing or coughing
3. Rapid heart rate
4. Anxiety
5. Coughing up blood
6. Swelling in the legs or arms
7. Fainting

Treatment for PE depends on the severity of the blockage and the overall health of the patient. Options include:

1. Anticoagulant medications to prevent clot formation and growth
2. Thrombolytic therapy, which involves administering drugs that dissolve blood clots
3. Embolectomy, a surgical procedure to remove the clot from the lung artery
4. Supportive measures such as oxygen therapy, fluid replacement, and pain management.

It's important to note that PE can be life-threatening if not treated promptly, so seeking medical attention right away is crucial.

--- Chain of Thought ---


...

Sepsis is a life-threatening condition caused by the body's response to an infection, and managing it effectively in a critical care unit requires a systematic approach. Here are the key steps in the sepsis management protocol:

1. Early recognition: Identify sepsis suspects based on clinical suspicion, using tools like the Quick Sequential Organ Failure Assessment (qSOFA) score or the Systemic Inflammatory Response Syndrome (SIRS) criteria.

2. Rapid diagnosis: Obtain blood cultures before administering antibiotics to ensure accurate identification of the causative organism. Other diagnostic tests, such as lactate levels and imaging studies, may also be helpful in confirming sepsis and identifying its source.

3. Immediate fluid resuscitation: Administer intravenous fluids to maintain adequate tissue perfusion and prevent hypotension. The goal is to achieve a mean arterial pressure (MAP) of 65 mmHg or higher.

4. Appropriate antibiotic therapy: Start broad-spectrum antibiotics based o

Llama.generate: prefix-match hit


Sepsis is when your body gets really sick from an infection, and we need to help it get better in the hospital's special unit called the Critical Care Unit (CCU). Here are some steps the doctors and nurses follow to take good care of you:

1. First, they will give you lots of fluids through a big IV to keep you hydrated and help your body fight the infection.
2. They might also give you medicines to help your body fight the infection and make you feel more comfortable.
3. The doctors and nurses will check your temperature, heart rate, breathing, and other vital signs very often to see how your body is responding to the treatments.
4. They may use machines like ventilators to help you breathe if you have trouble doing it on your own.
5. If needed, they might perform some tests like blood tests or imaging studies to understand the infection better and adjust the treatment plan accordingly.
6. The doctors and nurses will work together closely to manage your sepsis, and they may consult ot

Llama.generate: prefix-match hit


Sepsis is a life-threatening condition caused by the body's response to an infection, which can lead to tissue damage and organ failure. Management of sepsis in a critical care unit requires a multidisciplinary approach and prompt recognition and intervention to improve outcomes. Here are the key steps in managing sepsis in a critical care unit:

1. Early Recognition: The first step is to recognize sepsis early, ideally before it progresses to severe sepsis or septic shock. This involves assessing patients for signs and symptoms of infection, such as fever, chills, tachycardia, and altered mental status, and using clinical scoring systems like the Sequential Organ Failure Assessment (SOFA) score or the Quick Sequential Organ Failure Assessment (qSOFA) to identify those at risk.
2. Fluid Resuscitation: Once sepsis is suspected, prompt fluid resuscitation is essential to maintain adequate tissue perfusion and organ function. This involves administering intravenous fluids, such as normal 

### Observations (Prompt Engineering)
- **Temperature:** Low temperature (0.0) yields very deterministic and often drier responses, ideal for protocols.
- **Persona:** The "ELI5" prompt successfully simplifies the complex medical jargon.
- **Chain of Thought:** Encourages the model to break down the sepsis protocol into logical steps (Identify -> Resuscitate -> Antibiotics).

## Part 4: Data Preparation for RAG
Ingesting the Merck Manual PDF, chunking it, embedding it, and storing it in a Vector Database.

In [ ]:
import requests

doc_path = "014-NLP-PROJ-medical_diagnosis_manual_19.pdf"
github_url = "https://github.com/EvagAIML/014-NLP-Model-v1/blob/main/014-NLP-PROJ-medical_diagnosis_manual_19.pdf?raw=true"

# Download the PDF file from GitHub
if not os.path.exists(doc_path):
    print(f"Downloading {doc_path} from GitHub...")
    try:
        response = requests.get(github_url, stream=True)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        with open(doc_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print("Download complete.")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading file from GitHub: {e}")
        print("Please ensure the URL is correct and accessible.")

if os.path.exists(doc_path):
    loader = PyMuPDFLoader(doc_path)
    docs = loader.load()
    print(f"Loaded {len(docs)} pages.")

    # Check first 5 pages
    for i in range(min(5, len(docs))):
        print(f"--- Page {i+1} ---")
        print(docs[i].page_content[:500])
        print("...")
else:
    print(f"File {doc_path} not found even after attempted download. Please ensure it is in the working directory or the download path is correct.")

Loaded 4114 pages.
--- Page 1 ---
erikvdesigner@gmail.com
U36PAIRLB4
ant for personal use by erikvdesigner@g
shing the contents in part or full is liable
...
--- Page 2 ---
erikvdesigner@gmail.com
U36PAIRLB4
This file is meant for personal use by erikvdesigner@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.
...
--- Page 3 ---
Table of Contents
1
Front    ................................................................................................................................................................................................................
1
Cover    .......................................................................................................................................................................................................
2
Front Matter    .................................
...
--- Page 4 ---
491
Chapter 44. Foot & Ankle Disorders    ...............................................................

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
chunks = text_splitter.split_documents(docs)
print(f"Total Chunks created: {len(chunks)}")

Total Chunks created: 18020


In [ ]:
embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory="./chroma_db_medical_final"
)

# Default Retriever
retriever = vector_db.as_retriever(search_kwargs={"k": 3})

/tmp/ipython-input-30694839.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Part 5: Question Answering using RAG
We implement the RAG pipeline and fine-tune parameters to get the best evidence-based answers.

In [ ]:
def generate_rag_answer(query, k=3, temperature=0.1, max_tokens=512):
    # Retrieve
    retrieved_docs = vector_db.similarity_search(query, k=k)
    context = "\n\n".join([d.page_content for d in retrieved_docs])

    # Construct Prompt
    sys_msg = "You are a medical assistant. Use the following Context to answer the Question. If the answer is not in the context, say so."
    user_msg = f"Context:\n{context}\n\nQuestion: {query}"

    prompt = f"[INST] {sys_msg}\n{user_msg} [/INST]"

    # Generate
    output = llm(
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        stop=["[/INST]"]
    )
    return output['choices'][0]['text'].strip(), context

In [ ]:
# RAG Parameter Tuning (5 Combinations)
rag_configs = [
    {"name": "C1: Low Context, Precise", "k": 2, "temp": 0.0},
    {"name": "C2: Standard", "k": 3, "temp": 0.1},
    {"name": "C3: High Context", "k": 5, "temp": 0.1},
    {"name": "C4: Creative", "k": 3, "temp": 0.7},
    {"name": "C5: Max Context, Strict", "k": 7, "temp": 0.0}
]

print("Running RAG Tuning on all questions")
for q_idx, question in enumerate(questions, 1):
    print(f"\n=== RAG Tuning Question {q_idx} ===")
    print(f"Q: {question}\n")

    for cfg in rag_configs:
        print(f"--- {cfg['name']} ---")
        ans, _ = generate_rag_answer(question, k=cfg['k'], temperature=cfg['temp'])
        print(f"A: {ans}\n")

Running RAG Tuning on all questions

=== RAG Tuning Question 1 ===
Q: What are the common symptoms and treatments for pulmonary embolism?

--- C1: Low Context, Precise ---


Llama.generate: prefix-match hit


A: The common symptoms for pulmonary embolism include dyspnea (shortness of breath), pleuritic chest pain, cough, tachypnea (rapid breathing), tachycardia (fast heart rate), hypotension (low blood pressure), and a loud pulmonic component of the 2nd heart sound. The treatment for pulmonary embolism includes anticoagulants to prevent clot formation and growth, and in some cases, clot dissolution with thrombolytics or surgical removal. Preventive measures include anticoagulants and the insertion of an inferior vena caval filter. It's important to note that the context also mentions other conditions such as pneumonia or sepsis, foreign body embolism, and tumor embolism, which have different symptoms and treatments.

--- C2: Standard ---


Llama.generate: prefix-match hit


A: The common symptoms of pulmonary embolism include dyspnea (shortness of breath), pleuritic chest pain, cough, tachypnea (rapid breathing), tachycardia (fast heart rate), hypotension (low blood pressure), and a loud pulmonic component of the 2nd heart sound. The treatment for pulmonary embolism includes anticoagulants to prevent clot formation and growth, and in some cases, clot dissolution with thrombolytics or surgical removal. Preventive measures include anticoagulants and sometimes insertion of an inferior vena caval filter. It is important to note that the symptoms and treatments may vary depending on the severity and cause of the pulmonary embolism.

--- C3: High Context ---


...

A: The protocol for managing sepsis in a critical care unit involves several steps as outlined in the context:

1. Initial assessment and stabilization: Keeping the patient warm, controlling hemorrhage, checking airway and ventilation, providing respiratory assistance if necessary, and turning the patient's head to avoid aspiration if emesis occurs. Nothing should be given by mouth.
2. Simultaneous evaluation and treatment: Providing supplemental oxygen through a face mask, intubating and mechanically ventilating the patient if shock is severe or ventilation is inadequate, inserting two large IV catheters into separate peripheral veins, and providing an alternative access method such as a central venous line or intraosseous needle when peripheral veins cannot be promptly accessed.
3. Supportive care: Providing adequate nutrition, preventing infection, stress ulcers and gastritis, and pulmonary embolism.
4. Antibiotics: Administering empiric antibiotics after obtaining appropriate cultu

Llama.generate: prefix-match hit


A: The protocol for managing sepsis in a critical care unit involves several steps as outlined in the context, specifically in Chapters 222 and 227. Here are the essential steps:

1. Provide supportive care, including maintaining body warmth, controlling hemorrhage, securing the airway, and providing oxygen through a face mask or mechanical ventilation if necessary. Nothing should be given by mouth, and the patient's head should be turned to prevent aspiration if emesis occurs.
2. Begin treatment simultaneously with evaluation. Provide supplemental oxygen, intubate and mechanically ventilate if shock is severe or ventilation inadequate, and insert two large IV catheters into separate peripheral veins or use a central venous line or intraosseous needle if accessing peripheral veins promptly is not possible.
3. Provide aggressive fluid resuscitation to maintain adequate tissue perfusion.
4. Administer antibiotics as soon as possible to treat the underlying infection.
5. Perform surgical 

Llama.generate: prefix-match hit


A: The protocol for managing sepsis in a critical care unit involves several steps:

1. Recognition and diagnosis: If sepsis, severe sepsis, or septic shock is suspected based on symptoms such as shaking chills, persistent fever, altered sensorium, hypotension, and GI symptoms, cultures should be obtained of blood and any other appropriate specimens.
2. Initial resuscitation: First aid involves keeping the patient warm, controlling hemorrhage, checking and providing respiratory assistance if necessary, and avoiding anything by mouth. Supplemental O2 is provided, and airway intubation with mechanical ventilation may be necessary if shock is severe or if ventilation is inadequate. Two large IV catheters are inserted into separate peripheral veins, and a central venous line or intraosseous needle may be used as an alternative when peripheral veins cannot promptly be accessed.
3. Antibiotics: Empiric antibiotics are given after appropriate cultures are obtained, and early treatment with an

### Final RAG Answers (Best Configuration: k=3, temp=0.1)

In [ ]:
rag_responses = []
rag_contexts = []

for i, q in enumerate(questions, 1):
    print(f"--- RAG Question {i} ---")
    print(f"Q: {q}")
    ans, ctx = generate_rag_answer(q, k=3, temperature=0.1)
    print(f"A: {ans}\n")
    rag_responses.append(ans)
    rag_contexts.append(ctx)

--- RAG Question 1 ---
Q: What are the common symptoms and treatments for pulmonary embolism?


Llama.generate: prefix-match hit


A: The common symptoms of pulmonary embolism include dyspnea (shortness of breath), pleuritic chest pain, cough, tachypnea (rapid breathing), tachycardia (fast heart rate), hypotension (low blood pressure), and a loud pulmonic component of the 2nd heart sound. The treatment for pulmonary embolism includes anticoagulants to prevent clot formation or growth, and in some cases, clot dissolution with thrombolytics or surgical removal. Preventive measures include anticoagulants and sometimes insertion of an inferior vena caval filter. It is important to note that the symptoms and treatments may vary depending on the severity and cause of the pulmonary embolism.

--- RAG Question 2 ---
Q: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?


Llama.generate: prefix-match hit


A: The common symptoms of appendicitis include epigastric or periumbilical pain followed by brief nausea, vomiting, and anorexia. After a few hours, the pain shifts to the right lower quadrant. Pain increases with cough and motion. Classic signs are right lower quadrant direct and rebound tenderness located at McBurney's point (junction of the middle and outer thirds of the line joining the umbilicus to the anterior superior spine). Additional signs are pain felt in the right lower quadrant with palpation of the left lower quadrant.

Appendicitis cannot be cured via medicine alone, as it requires surgical removal of the inflamed appendix. The standard surgical procedure for treating appendicitis is an appendectomy, which can be performed laparoscopically or open depending on the severity and complications of the condition. In some cases, if an abscess or an inflammatory mass has formed, the procedure may be limited to drainage of the abscess before performing an appendectomy.

--- RAG 

...

A: For a person who has fractured their leg during a hiking trip, the following precautions and treatment steps are necessary:

1. Immediate immobilization: The leg should be immobilized as soon as possible to prevent further injury to soft tissues. This can be done by using a splint or a makeshift device until professional help arrives.
2. Seek medical attention: It is essential to seek medical attention as soon as possible, especially if the fracture is severe or open (in communication with a skin wound). The person may require surgical intervention for artery repair or nerve repair, depending on the extent of the injury.
3. Hemorrhage control: If there is significant bleeding, attempts should be made to control it by applying pressure to the wound with a clean cloth or dressing.
4. Transportation: The person should be transported to the nearest medical facility as soon as possible. If they are unable to walk, they may need to be carried or evacuated using a stretcher or other means.

Llama.generate: prefix-match hit


A: The diagnostic steps for suspected endocrine disorders include:

1. Measuring levels of peripheral endocrine hormones and pituitary hormones in the blood.
2. Detecting autoantibodies to each affected glandular tissue to differentiate Primary Disorders of Specific Hormones (PDS) from other causes.
3. Elevated levels of pituitary tropic hormones suggest that the hypothalamic-pituitary axis is intact, although some patients with type II PDS may have hypothalamic-pituitary insufficiency.
4. Lifelong follow-up is necessary as symptoms can begin insidiously and be nonspecific, and unrecognized endocrine deficiencies such as hypoparathyroidism or adrenal insufficiency can be life-threatening.
5. Relatives should be made aware of the diagnosis and screened when appropriate, and measurement of glutamic acid decarboxylase antibodies may be useful in determining risk.

--- RAG Question 9 ---
Q: What is the protocol for managing sepsis in a critical care unit?


Llama.generate: prefix-match hit


A: The protocol for managing sepsis in a critical care unit involves several steps as outlined in the context:

1. Provide supportive care, including maintaining body warmth, controlling hemorrhage, checking airway and ventilation, and providing respiratory assistance if necessary. Nothing should be given by mouth, and the patient's head should be turned to one side to avoid aspiration if emesis occurs.
2. Begin treatment simultaneously with evaluation. Provide supplemental oxygen via face mask, and if shock is severe or if ventilation is inadequate, perform airway intubation with mechanical ventilation. Insert two large IV catheters into separate peripheral veins, and consider a central venous line or an intraosseous needle as an alternative when peripheral veins cannot be promptly accessed.
3. Provide aggressive fluid resuscitation to maintain adequate tissue perfusion.
4. Administer antibiotics to address the underlying bacterial infection.
5. Perform surgical excision of infected o

### Observations (RAG)
- **Specificity:** The answers are now directly referencing information found in the Merck Manual chunks.
- **Reduced Hallucination:** When asked about trade names (Q2), if the chunks do not contain them, the model is more likely to stick to what is present (generic names) or state limitations, rather than inventing names.
- **Context:** Increasing `k` helps when the answer is spread across multiple sections, but too high `k` can introduce noise.

## Part 6: Output Evaluation (LLM-as-a-Judge)
Evaluating the RAG responses for **Groundedness** and **Relevance**.

In [ ]:
def evaluate_response(query, answer, context):
    # Groundedness Prompt
    ground_prompt = f"""
    [INST] You are a judge. Determine if the Answer is fully supported by the Context.
    Context: {context}
    Answer: {answer}
    Reply with 'YES' if supported, 'NO' if not.
    [/INST]
    """
    # Use Judge LLM
    ground_score = judge_llm(ground_prompt, max_tokens=10, temperature=0.0)['choices'][0]['text'].strip()

    # Relevance Prompt
    rel_prompt = f"""
    [INST] You are a judge. Determine if the Answer directly addresses the Question.
    Question: {query}
    Answer: {answer}
    Reply with 'YES' if relevant, 'NO' if not.
    [/INST]
    """
    # Use Judge LLM
    rel_score = judge_llm(rel_prompt, max_tokens=10, temperature=0.0)['choices'][0]['text'].strip()

    return ground_score, rel_score

print("--- Evaluation Results ---\n")
for i, (q, a, c) in enumerate(zip(questions, rag_responses, rag_contexts), 1):
    g, r = evaluate_response(q, a, c)
    print(f"Q{i}: Groundedness: {g} | Relevance: {r}")

--- Evaluation Results ---



Llama.generate: prefix-match hit


Q1: Groundedness:  | Relevance: [QUESTION] What are the common


Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Q2: Groundedness:  | Relevance: 


...

Q7: Groundedness: [USERBOT] Can you provide a summary | Relevance: 


Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Q8: Groundedness:  | Relevance: 


Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Q9: Groundedness: Chapter 230. Septic | Relevance: 


## Part 7: Actionable Insights and Recommendations

### Key Takeaways
1. **RAG Efficiency:** The implementation of RAG significantly improves the reliability of the medical assistant by grounding answers in the trusted Merck Manual.
2. **Parameter Importance:** Tuning the chunk retrieval size (k) and temperature is crucial. `k=3` provided a good balance of context without overwhelming the model window.
3. **Prompt Engineering:** Specifying a persona (e.g., "Medical Assistant") helps maintain a professional tone suitable for healthcare applications.
4. **Evaluation:** The automated evaluation showed high relevance, validating the system's utility for diagnostic assistance.

### Business Recommendations
- **Deployment:** Deploy this prototype to a pilot group of clinicians to assist with information retrieval.
- **Data Expansion:** Integrate more specialized journals apart from the Merck Manual for wider coverage.
- **User Interface:** Build a frontend (Streamlit/React) to allow doctors to query easily from tablets.